# LLM Functions

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

In [ ]:
SELECT * FROM SUPPORT_TICKETS;

## CLASSIFY_TEXT

Categorize the text in to the one of the categories provided.

In [ ]:
SELECT request as text, ARRAY_CONSTRUCT('Internet','Phone') as classes,
       SNOWFLAKE.CORTEX.CLASSIFY_TEXT(text,classes)['label'] as classification
FROM support_tickets LIMIT 5;

## COMPLETE

### Supported Models

`gemma-7b`,`jamba-1.5-mini`,`jamba-instruct`,`llama2-70b-chat`,`llama3-8b`

`llama3-70b`,`llama3.1-8b`,`llama3.1-70b`, `llama3.1-405b`, 

`mistral-large `, `mistral-large2`, `mistral-7b`, `mixtral-8x7b`

`reka-core`, `reka-flash` ,`snowflake-arctic`



In [ ]:
SELECT request as text, 
   SNOWFLAKE.CORTEX.COMPLETE(
     'snowflake-arctic',
      CONCAT('Summarize this request in bullet points: <request>', text, '</request>')) as issue_summary
  FROM SUPPORT_TICKETS LIMIT 1;

In [ ]:
complete_df = complete.to_df().to_pandas()
st.write(complete_df['TEXT'][0])
st.write(complete_df['ISSUE_SUMMARY'][0])

## SENTIMENT

In [ ]:
SELECT REQUEST, ROUND(SNOWFLAKE.CORTEX.SENTIMENT(REQUEST))::INT AS Sentiment
FROM SUPPORT_TICKETS
LIMIT 10;

In [ ]:
st.markdown(
    """
- :blush: : :green[Positive]
- :neutral_face: :  :orange[Neutral]
- :disappointed:  : :red[Negative]
"""
)
df = sentiment_analysis.to_df().to_pandas()
df = df.astype('string')
# when sentiment score is -1 then its negative
df.loc[df['SENTIMENT'] == "-1", 'SENTIMENT'] = "😞"
# when sentiment score is 0 then its neutral
df.loc[df['SENTIMENT'] == "0", 'SENTIMENT'] = "😐"
# when sentiment score is 1 then its positive
df.loc[df['SENTIMENT'] == "1", 'SENTIMENT'] = "😊"
st.dataframe(df)

## SUMMARIZE

In [ ]:
SELECT REQUEST,SNOWFLAKE.CORTEX.SUMMARIZE(REQUEST) AS Summary
FROM SUPPORT_TICKETS
LIMIT 1;

## PROMPTING

In [ ]:
SET  LLM_PROMPT='Summarize this request in less than 200 words.Put the fees, request and country in JSON format.';
SELECT request,SNOWFLAKE.CORTEX.COMPLETE('snowflake-arctic',CONCAT('[INST]',$LLM_PROMPT,request,'[/INST]')) AS Summary
FROM SUPPORT_TICKETS
LIMIT 1;

In [ ]:
df = json_summary.to_df().to_pandas()
st.json(df.iloc[0,1])

## TRANSLATE

### Supported languages 

| Language | Code 
| :---: | :--:
English | 'en'
French | 'fr'
German |'de'
Italian |'it'
Japanese | 'ja'
Korean | 'ko'
Polish | 'pl'
Portuguese | 'pt'
Russian |'ru'
Spanish | 'es'
Swedish |'sv'

In [ ]:
SELECT request as text, SNOWFLAKE.CORTEX.TRANSLATE(text,'en','fr') as translated FROM SUPPORT_TICKETS LIMIT 1;

In [ ]:
df = translate.to_df().to_pandas()
st.write(df["TEXT"][0])
st.subheader("Translation")
st.write(df["TRANSLATED"][0])

## EXTRACT_ANSWER

In [ ]:
SELECT SNOWFLAKE.CORTEX.EXTRACT_ANSWER(request,
    'How much roaming fee charged?')[0]['answer']::STRING as roaming_fees
FROM support_tickets LIMIT 3;

## Document AI

In [ ]:
WITH invoice_info as  ( SELECT KAMESH_LLM_DEMO.DOC_AI.INVOICE_INFO!PREDICT(
  GET_PRESIGNED_URL(@DOCS, 'Invoice 9987.pdf'), 1) as invoice_details
)
SELECT * from invoice_info;

In [ ]:
import json
from re import sub
from decimal import Decimal

df = invoice_details.to_df().to_pandas()
out = json.loads(df.iloc[0,0])
extracted_data = { key: item[0]['value'] for key, item in out.items() if key != "__documentMetadata"}
df = pd.DataFrame([extracted_data])
df = df.rename(columns={
    "CUSTOMER_NAME_ADDR": "Customer Name and Address",
    "DUE_DATE": "Due Date",
    "INTERNATIONAL_ROAMING_FEE": "Intl. Roaming Fee",
    "INVOICE_NUMBER": "Invoice Number",
    "PO_NUMBER": "Purchase Order",
    "SERVICE_PLAN": "Plan",
    "TOTAL_DUE": "Due",
})
tbl = df.loc[:,["Invoice Number","Purchase Order","Customer Name and Address","Plan","Due Date"]]
st.header("Invoice Details")
st.dataframe(tbl,hide_index=True)
col1, col2 = st.columns(2)
due = df.loc[0,"Due"]
roaming = df.iloc[0,2]
col1.metric("International Roaming",roaming)
romaing_contrib =  ( Decimal(sub(r'[^\d.]', '', roaming)) / Decimal(sub(r'[^\d.]', '', due)) ) * 100
col2.metric("Total",due,delta=f"{romaing_contrib:0.2f}% is roaming")

